# Evaluating the Operations Rehearsal for Commissioning simulated observing sequence

In [ ]:
import pandas as pd
import numpy as np
from astropy.time import Time
import sqlite3

In [ ]:
import lsst.daf.butler as dafButler
from lsst.pipe.tasks.loadReferenceCatalog import LoadReferenceCatalogConfig, LoadReferenceCatalogTask
from lsst.utils import getPackageDir
import lsst.geom

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
db = 'ops_rehearsal_apr_2024.db'

In [ ]:
conn = sqlite3.connect(db)

In [ ]:
observations = pd.read_sql('select * from observations;', conn)

In [ ]:
observations

In [ ]:
observations.columns

In [ ]:
np.unique(observations['note'])

In [ ]:
# Note that can only set this date after downloading the full set sky brightness data
#mjd_start = Time('2024-04-01 00:00:00.000', format='iso').mjd
time_start = Time(np.min(observations['mjd']), format='mjd')
time_stop = Time(np.max(observations['mjd']), format='mjd')
print(time_start.iso)
print(time_stop.iso)

In [ ]:
# Identify first night of the rehearsal
time_first = Time('2024-04-03 00:00:00.000', format='iso')
index_first = np.argmin(np.absolute(time_first.mjd - observations['mjd']))
#print(time_first.mjd)
#print(observations['mjd'][index_first])
night_first = observations['night'][index_first]
print(night_first)
selection = (observations['night'] == night_first)
print(
    Time(np.min(observations['mjd'][selection]), format='mjd').iso
)
print(
    Time(np.max(observations['mjd'][selection]), format='mjd').iso
)
plt.figure()
plt.plot(observations['mjd'][selection], np.degrees(observations['sunAlt'][selection]))

## Visualize Single Field

In [ ]:
for survey_name in np.unique(observations['note']):
    print(survey_name, np.sum(observations['note'] == survey_name))

In [ ]:
survey_name = 'Rubin_SV_225_-40'
filter = 'r'

In [ ]:
selection = (observations['note'] == survey_name) & (observations['filter'] == filter)
ra_boresight_array = np.degrees(observations['RA'][selection])
dec_boresight_array = np.degrees(observations['dec'][selection])
rotation_array = np.degrees(observations['rotSkyPos'][selection])

In [ ]:
plt.figure()
plt.scatter(ra_boresight_array, dec_boresight_array, c=rotation_array)
plt.colorbar(label='Rotation (deg)')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
plt.suptitle('Telescope Boresight Pointings and Camera Rotation Angles')

In [ ]:
plt.figure()
plt.hist(rotation_array, bins=41)

## Visualize Single Field during Rehearsal Time Window

In [ ]:
survey_name = 'Rubin_SV_225_-40'
filter = 'r'
nights_rehearsal = [night_first, night_first + 1, night_first + 2]

#selection = (observations['note'] == survey_name) & (observations['filter'] == filter) & (observations['night'] == night_first)
selection = (observations['note'] == survey_name) & (observations['filter'] == filter) & np.in1d(observations['night'], nights_rehearsal)
ra_boresight_array = np.degrees(observations['RA'][selection])
dec_boresight_array = np.degrees(observations['dec'][selection])
rotation_array = np.degrees(observations['rotSkyPos'][selection])

print(np.sum(selection))

In [ ]:
plt.figure()
plt.scatter(ra_boresight_array, dec_boresight_array, c=rotation_array)
plt.colorbar(label='Rotation (deg)')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
plt.suptitle('Telescope Boresight Pointings and Camera Rotation Angles')

## Visualize Survey

In [ ]:
print(np.sum(observations['airmass'] > 2.0))
print(np.max(observations['airmass']))

In [ ]:
print(np.min(np.degrees(observations['moonDist'])))

In [ ]:
np.unique(observations['note'])

In [ ]:
np.unique(observations['filter'])

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['alt'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['slewtime'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
plt.scatter(observations['mjd'], np.degrees(observations['rotSkyPos']), c=np.degrees(observations['RA']), s=1)
plt.colorbar(label='RA (deg)')

plt.xlabel('MJD')
plt.ylabel('rotSkyPos')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
#plt.scatter(observations['mjd'], np.degrees(observations['rotTelPos']), c=np.degrees(observations['RA']), s=1)
#plt.colorbar(label='RA (deg)')

for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['rotTelPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()

plt.xlabel('MJD')
plt.ylabel('rotTelPos')
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['RA'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('MJD')
plt.ylabel('RA (deg)')

In [ ]:
plt.figure(dpi=200)
#for filtername in f2c:
#    in_filt = np.where(observations['filter'] == filtername)[0]
#    if in_filt.size > 0:
#        plt.plot(observations['mjd'][in_filt], np.degrees(observations['moonDist'][in_filt]), 
#                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.scatter(observations['mjd'], np.degrees(observations['moonDist']), c=observations['moonPhase'], s=1)
plt.colorbar(label='moonPhase')

#plt.legend()
plt.xlabel('MJD')
plt.ylabel('moonDist (deg)')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), np.degrees(observations['rotSkyPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('rotSkyPos')

In [ ]:
gap = (np.diff(observations['mjd']) * 24. * 3600.)

plt.figure()
plt.yscale('log')
bins = np.linspace(0., 300., 101)
plt.hist(gap, bins=bins)
plt.ylabel('Counts')
plt.xlabel('Time Gap (s)')

selection = gap > 300.
for index in np.nonzero(selection)[0]:
    print(gap[index], observations['mjd'][index + 1])

# Reference Catalog

In [ ]:
# https://developer.lsst.io/usdf/storage.html#butler-access
repo = '/sdf/group/rubin/repo/main_20210215/'

In [ ]:
config = os.path.join(repo, 'butler.yaml')
butler = dafButler.Butler(config=config)
registry = butler.registry

In [ ]:
collection = 'refcats'
registry.getCollectionSummary(collection).dataset_types.names

In [ ]:
#refDataset = 'gaia_dr3'
refDataset = 'gaia_dr3_20230707'

In [ ]:
ra_center = np.median(ra_boresight_array)
dec_center = np.median(dec_boresight_array)
print(ra_center, dec_center)

In [ ]:
# ra_center, dec_center = 180., -1.

In [ ]:
import lsst.sphgeom

In [ ]:
level = 7  # the resolution of the HTM grid
pixelization = lsst.sphgeom.HtmPixelization(level)

In [ ]:
circle = lsst.sphgeom.Circle(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(ra_center, dec_center)
    ),
    lsst.sphgeom.Angle.fromDegrees(4.)
)

In [ ]:
htm_set = np.concatenate([list(range(_[0], _[1])) for _ in pixelization.envelope(circle)]).tolist()

In [ ]:
len(htm_set)

In [ ]:
where_htm = "htm7 in (%s)"%(", ".join(str(_) for _ in htm_set))
print(where_htm)

In [ ]:
datasetRefs = list(registry.queryDatasets(datasetType=refDataset, where=where_htm, collections=collection).expanded())

In [ ]:
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(ra_center, dec_center)
    )
)
print(htm_id)

In [ ]:
circle = pixelization.triangle(htm_id).getBoundingCircle()
scale = circle.getOpeningAngle().asDegrees()*3600.
level = pixelization.getLevel()
print(f'HTM ID={htm_id} at level={level} is bounded by a circle of radius ~{scale:0.2f} arcsec.')

In [ ]:
type(pixelization.triangle(htm_id).getBoundingCircle())

In [ ]:
help(pixelization.index)

In [ ]:
# This is what I tried queryDatasets(..., collections="refcats", where=htm_where, findFirst=True), with htm_where="htm7 in (..list_of_shards..)"

In [ ]:
# help(lsst.sphgeom.HtmPixelization)

Arbitrary regions aren't supported directly, but you can use something like "htm7 IN (A..B)" where I wrote "skymap='hsc_rings_v1' AND tract=9813" to do a query over lists of HTM IDs at some level, and lsst.sphgeom.Circle and lsst.sphgeom.HtmPixelization can be used to get those from a circle on the sky (happy to provide an example if that's what you want to do; also note that the C++ docs for these are decent, while the Python docs are nonexistent - but the interfaces are a straightforward translation).

In [ ]:
config = LoadReferenceCatalogConfig()
config.refObjLoader.anyFilterMapsToThis = 'phot_g_mean'
config.doApplyColorTerms = False

In [ ]:
# Alternative not recommended approach because only gets a single htm trixel
# datasetRefs = list(registry.queryDatasets(datasetType=refDataset, htm3=htm_id, collections=collection).expanded())

In [ ]:
dataIds = [_.dataId for _ in datasetRefs]

In [ ]:
refCats = [butler.getDeferred(refDataset, _, collections=['refcats']) for _ in dataIds]

In [ ]:
loaderTask = LoadReferenceCatalogTask(config=config, name=refDataset, dataIds=dataIds, refCats=refCats)

In [ ]:
center = lsst.geom.SpherePoint(ra_center,
                               dec_center,
                               lsst.geom.degrees)

In [ ]:
# 10 seconds
refCatSpatial = loaderTask.getSkyCircleCatalog(center,
                                               1.*lsst.geom.degrees,
                                               ['i'])

In [ ]:
len(refCatSpatial)

In [ ]:
plt.figure()
plt.scatter(refCatSpatial['ra'], refCatSpatial['dec'], s=1)
plt.xlim(plt.xlim()[::-1])

# Transformation for sky to camera coordinates

In [ ]:
from lsst.afw.cameraGeom import Camera

In [ ]:
#help(Camera)

In [ ]:
repo = '/sdf/group/rubin/repo/dc2_20210215'
collection = '2.2i/runs/DP0.1'

In [ ]:
butler = dafButler.Butler(repo, collections=collection)
camera = butler.get('camera', instrument='LSSTCam-imSim')

In [ ]:
#camera

In [ ]:
#dir(camera)

In [ ]:
import lsst.afw.geom as afwGeom
import lsst.afw.cameraGeom as cameraGeom

In [ ]:
# https://lsstc.slack.com/archives/C2JPMCF5X/p1569266787124100
# https://community.lsst.org/t/constructing-a-wcs-from-camera-geometry/3039
def getWcsFromDetector(detector, boresight, rotation=0*lsst.geom.degrees, flipX=False):
    """Given a detector and (boresight, rotation), return that detector's WCS

    Parameters
    ----------
    detector : `lsst.afw.cameraGeom.Detector`
        A detector in a camera.
    boresight : `lsst.geom.SpherePoint`
       The boresight of the observation.
    rotation : `lsst.afw.geom.Angle`, optional
        The rotation angle of the camera.
        The rotation is "rotskypos", the angle of sky relative to camera
        coordinates (from North over East).
    flipX : `bool`, optional
        Flip the X axis?

    Returns
    -------
    wcs : `lsst::afw::geom::SkyWcs`
        The calculated WCS.
    """
    trans = detector.getTransform(detector.makeCameraSys(cameraGeom.PIXELS),
                                  detector.makeCameraSys(cameraGeom.FIELD_ANGLE))

    wcs = afwGeom.makeSkyWcs(trans, rotation, flipX, boresight)

    return wcs

In [ ]:
def getDetectorId(refCatSpatial, camera, boresight, rotation, flipX=False):
    detId = np.zeros(len(refCatSpatial)) - 1.
    for det in camera:
        
        # Emulate ComCam by considering only the central raft
        if 'R22' not in det.getName():
            continue
            
        wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
        x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
        
        # Detector
        selected = det.getBBox().contains(x, y)
        detId[selected] = det.getId()
        #print(det.getId(), det.getName(), np.sum(selected))
        
        # Focal Plane Coordinates
        #xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
        #x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])

    #return detId, np.array(x_fp), np.array(y_fp)
    #return detId, x_fp, y_fp
    return detId

In [ ]:
def getFpCoordinates(refCatSpatial, camera, boresight, rotation, flipX=False):
    # Emulate ComCam
    det = camera.get('R22_S11')
    wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
    x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
    xy_fp = det.transform([lsst.geom.Point2D(x, y) for x, y in zip(x, y)], cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    x_fp, y_fp = zip(*[[_.x, _.y] for _ in xy_fp])
    return np.array(x_fp), np.array(y_fp)

## Test transform from sky to camera coordinates

In [ ]:
boresight = lsst.geom.SpherePoint(180., -1., units=lsst.geom.degrees)
rotation = 45.*lsst.geom.degrees
flipX = False

In [ ]:
#for det in camera:
#    center = det.getOrientation().getFpPosition()
#    print(center)

In [ ]:
det = camera[0]

In [ ]:
det.getName()

In [ ]:
#dir(det)

In [ ]:
trans = det.getTransform(det.makeCameraSys(cameraGeom.PIXELS), det.makeCameraSys(cameraGeom.FIELD_ANGLE))

In [ ]:
wcs = afwGeom.makeSkyWcs(trans, rotation, flipX, boresight)

In [ ]:
wcs

In [ ]:
wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)

In [ ]:
help(wcs.skyToPixelArray)

In [ ]:
x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)

In [ ]:
plt.figure()
plt.scatter(x, y)

In [ ]:
selected = det.getBBox().contains(x, y)

In [ ]:
plt.figure()
plt.scatter(x[~selected], y[~selected], s=1)
plt.scatter(x[selected], y[selected], s=1)

In [ ]:
det.getId()

In [ ]:
#detId = np.empty(len(refCatSpatial)).fill(-1.)
detId = np.zeros(len(refCatSpatial)) - 1.
for det in camera:
    if 'R22' not in det.getName():
        continue
    wcs = getWcsFromDetector(det, boresight, rotation=rotation, flipX=flipX)
    x, y = wcs.skyToPixelArray(refCatSpatial['ra'], refCatSpatial['dec'], degrees=True)
    selected = det.getBBox().contains(x, y)
    detId[selected] = det.getId()
    print(det.getId(), det.getName(), np.sum(selected))

In [ ]:
detId = getDetectorId(refCatSpatial, camera, boresight, rotation, flipX)
#detId, x_fp, y_fp = getDetectorId(refCatSpatial, camera, boresight, rotation, flipX)

In [ ]:
x_fp, y_fp = getFpCoordinates(refCatSpatial, camera, boresight, rotation, flipX)

In [ ]:
np.unique(detId)

In [ ]:
selected = (detId >= 0)

plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=detId[selected], s=1)
plt.colorbar()

In [ ]:
plt.figure()
plt.scatter(x_fp[selected], y_fp[selected], c=detId[selected], s=1)
plt.colorbar()

In [ ]:
detId = np.zeros(len(refCatSpatial)) - 1
print(detId.shape)
print(detId)

In [ ]:
detId = np.empty(len(refCatSpatial))
print(detId.shape)
print(detId)

# Set of dithers

In [ ]:
# For testing
# n = 20
# ra_boresight_array = 180. + np.random.uniform(-0.2, 0.2, size=n)
# dec_boresight_array = -1. + np.random.uniform(-0.2, 0.2, size=n)
# rotation_array = np.random.uniform(-180., 180., size=n)

In [ ]:
survey_name = 'Rubin_SV_225_-40'
filter = 'r'
nights_rehearsal = [night_first, night_first + 1, night_first + 2]
#nights_rehearsal = [night_first]

#selection = (observations['note'] == survey_name) & (observations['filter'] == filter) & (observations['night'] == night_first)
selection = (observations['note'] == survey_name) & (observations['filter'] == filter) & np.in1d(observations['night'], nights_rehearsal)
ra_boresight_array = np.degrees(observations['RA'].values[selection])
dec_boresight_array = np.degrees(observations['dec'].values[selection])
rotation_array = np.degrees(observations['rotSkyPos'].values[selection])

print(np.sum(selection))

In [ ]:
# Subsample
ra_boresight_array = ra_boresight_array[::6]
dec_boresight_array = dec_boresight_array[::6]
rotation_array = rotation_array[::6]

print(len(ra_boresight_array))

In [ ]:
plt.figure()
plt.scatter(ra_boresight_array, dec_boresight_array, c=rotation_array)
plt.scatter(ra_center, dec_center, c='red', marker='s')
plt.colorbar(label='Rotation (deg)')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
plt.suptitle('Telescope Boresight Pointings and Camera Rotation Angles')

In [ ]:
n = 30
detId_array = []
x_fp_array = []
y_fp_array = []
for visit_index in range(0, n):
    print(visit_index)
    boresight_visit = lsst.geom.SpherePoint(ra_boresight_array[visit_index], dec_boresight_array[visit_index], units=lsst.geom.degrees)
    rotation_visit = rotation_array[visit_index]*lsst.geom.degrees
    detId = getDetectorId(refCatSpatial, camera, boresight_visit, rotation_visit)
    detId_array.append(detId)
    x_fp, y_fp = getFpCoordinates(refCatSpatial, camera, boresight_visit, rotation_visit)
    x_fp_array.append(x_fp)
    y_fp_array.append(y_fp)

In [ ]:
detId_array = np.array(detId_array)
x_fp_array = np.array(x_fp_array)
y_fp_array = np.array(y_fp_array)

In [ ]:
detId_array.shape

In [ ]:
counts = np.sum(detId_array >= 0, axis=0)

In [ ]:
selected = (counts > 0)
plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=counts[selected], s=1)
plt.colorbar(label='Number of Measurements')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')

In [ ]:
plt.figure()
plt.plot(np.sort(counts[selected])[::-1])
plt.xlabel('Number of Reference Stars')
plt.ylabel('Number of Measurements')

In [ ]:
# Number of unique detectors that each star appeared on

In [ ]:
unique_det = np.array([len(np.unique(_[_ >= 0])) for _ in detId_array.T])

In [ ]:
selected = (counts >= 1)
plt.figure()
plt.scatter(refCatSpatial['ra'][selected], refCatSpatial['dec'][selected], c=unique_det[selected], s=1)
plt.colorbar(label='Number of Unique Detectors')
plt.xlim(plt.xlim()[::-1])
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')

In [ ]:
plt.figure()
plt.plot(np.sort(unique_det[selected])[::-1])
plt.xlabel('Number of Reference Stars')
plt.ylabel('Number of Unique Detectors')

In [ ]:
# Number of repeated visits in focal plane coordinates
# Need to define some minimum number of measurements per star

In [ ]:
counts_array = np.tile(counts, detId_array.shape[0]).reshape(detId_array.shape)
unique_det_array = np.tile(unique_det, detId_array.shape[0]).reshape(detId_array.shape)

In [ ]:
plt.figure()
plt.scatter(x_fp_array[detId_array >= 0], y_fp_array[detId_array >= 0], c=unique_det_array[detId_array >= 0], s=1)
plt.colorbar(label='Number of Unique Detectors')
plt.xlabel('Focal Plane x (mm)')
plt.ylabel('Focal Plane y (mm)')

In [ ]:
plt.figure()
plt.scatter(x_fp_array[detId_array >= 0], y_fp_array[detId_array >= 0], c=counts_array[detId_array >= 0], s=1)
plt.colorbar(label='Number of Measurements')
plt.xlabel('Focal Plane x (mm)')
plt.ylabel('Focal Plane y (mm)')